<a href="https://colab.research.google.com/github/ykitaguchi77/chatGPT/blob/main/OPENAI_agent_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os
from google.colab import userdata

# OpenAI APIキーをColabのシークレットから取得して設定
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key

print("APIキーが設定されました")

def set_api():
  os.environ["OPENAI_API_KEY"] = api_key


APIキーが設定されました


In [11]:
!pip install openai-agents

Code calls on itself,  
Layers deep, solutions hide—  
Infinite returns.


In [34]:
from agents import Agent, Runner

set_api()

agent = Agent(name="Assistant", instructions="あなたは面白い漫才師です")

# Intended for Jupyter notebooks where there's an existing event loop
result = await Runner.run(agent, "面白いダジャレを言って下さい.") # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)


分かりました！こんなのはどうでしょう：

靴屋が言った、「今日も歩くついでに一足おくるよ！」

どうでしょう、足を踏み出すたびに笑えるかな？


In [36]:
from agents import Agent, Runner
import asyncio

set_api()

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)


async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)
    # ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?


if __name__ == "__main__":
    asyncio.run(main())

¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás?
